In [1]:
# Linear V4: SVM, TERd
# Linear V3 : Normal, SVM

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve
from sklearn.svm import SVC
from keras.utils import to_categorical
import karnet_v1
from karnet_v1 import KARnet

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\herok\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\herok\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\herok\Anaconda3\lib\site-packages\tensorflow\python\framework\

In [2]:
from siam_utils import *
import siamkar_v2_hp
import karnet_v1

Instructions for updating:
Colocations handled automatically by placer.


In [29]:
## Functions
# function to return key for any value 
def get_key(val,my_dict): 
    for key, list_val in my_dict.items(): 
         if val in list_val: 
             return key 
  
    return "key doesn't exist"

def ReadCSIData(PATH,ratio,locs,dirs):
    print("loading data from {}".format(PATH))
    list_csi = []
    list_lab = []
    list_res_csi = []
    list_res_lab = []
    
    ids = int(ratio * 100)
    res_ids = int((1-ratio)*100)

    # Filter Loc,Dir
    file_list = []
    for i in locs:
        for j in dirs:
            filename = 'Dataset_' + str(i) + '_' + str(j) + '.npy'
            file_list.append(filename)

    for file in file_list:
        data_read = np.load(PATH + file)
        csi_read = data_read[:,4:].astype('float32')
        lab_read = data_read[:,0].astype('int')

        data_x = csi_read.reshape([-1,10,6,30,500]).swapaxes(2,4)

        uniq_label = np.unique(lab_read)
        label_table = pd.Series(np.arange(len(uniq_label)),index=uniq_label)
        data_y = np.array([label_table[num] for num in lab_read]).reshape([-1,10])
        
        # use half of the dataset
        idx_ids = (data_y[:,0] < ids)
        list_csi.append(data_x[idx_ids])
        list_lab.append(data_y[idx_ids])
        
        reserve_ids = (data_y[:,0] >= ids)
        list_res_csi.append(data_x[reserve_ids])
        list_res_lab.append(data_y[reserve_ids])

    arr_csi = np.array(list_csi).swapaxes(0,1).reshape([ids*len(file_list),10,500,30,6])
    arr_lab = np.array(list_lab).swapaxes(0,1).reshape([ids*len(file_list),10])
    Xs = arr_csi.reshape([-1,1,500,30,6])
    Ys = arr_lab.reshape([-1,1])

    reserve_csi = np.array(list_res_csi).swapaxes(0,1).reshape([res_ids*len(file_list),10,500,30,6])
    reserve_lab = np.array(list_res_lab).swapaxes(0,1).reshape([res_ids*len(file_list),10])
    reserve_Xs = reserve_csi.reshape([-1,1,500,30,6])
    reserve_Ys = reserve_lab.reshape([-1,1])

    return([Xs,Ys,reserve_Xs,reserve_Ys])

class LinearModels:
    isdual = 0 #prim:0,dual:1
    model_name = 'Default'
    train_time = 0

    def __init__(self,data):
        self.X = data[0]
        self.Y = data[1]
        self.Xval = data[2]
        self.Yval = data[3]
    
    def model(self):
        if self.isdual:
            return(np.eye(X.shape[0],X.shape[0]))
        else:
            return(np.eye(X.shape[1],X.shape[1]))

    def train(self):
        t1 = time.time()
        alpha_mat = self.model()
        LinearModels.train_time = time.time()-t1
        print("Training Time: " + str(LinearModels.train_time))
        return(alpha_mat)

    def val_dist(self,alpha_mat):
        if self.isdual:
            val_mat = np.dot(self.Xval,self.X.T).dot(alpha_mat)
        else:
            val_mat = self.Xval.dot(alpha_mat)

        res_list = []
        for i in range(val_mat.shape[0]):
            for ti in range(val_mat.shape[0]):
                val_dist = val_mat[i,:] - val_mat[ti,:]
                val_score = np.sqrt(np.sum(np.multiply(val_dist,val_dist)))
                grd_truth =  int(self.Yval[i] == self.Yval[ti])
                score_list = [i,ti,grd_truth,val_score]
                res_list.append(score_list)
        
        res_df = pd.DataFrame(res_list,columns=['i','ti','truth','score'])
        #return(res_df[res_df.i != res_df.ti])
        return(val_mat)
    
    def eer_graphs(self,dist_truth,dist_score,pos_label):
        fpr, tpr, thresholds = roc_curve(dist_truth, dist_score,pos_label=pos_label)
        eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
        
        # ROC
        plt.plot(fpr, tpr, '.-', label=self.model_name)
        plt.plot([0, 1], [0, 1], 'k--', label="random guess")
        
        plt.xlabel('False Positive Rate (Fall-Out)')
        plt.ylabel('True Positive Rate (Recall)')
        plt.title('Receiver Operating Characteristic curve')
        plt.legend()
        plt.show()
        
        # EER
        thres_mask = thresholds <= 5*np.median(dist_score)
        
        th_mask = thresholds[thres_mask]
        fpr_mask= fpr[thres_mask]
        tpr_mask = tpr[thres_mask]
        
        plt.plot(th_mask,fpr_mask,color='blue', label="FPR")
        plt.plot(th_mask,1-tpr_mask,color='red',label="FNR")
        plt.plot(th_mask,fpr_mask + (1-tpr_mask),color='green',label="TER")
        plt.axhline(eer,color='black')
        plt.text(max(th_mask)*1.05,eer,'EER : ' + str(round(eer*100,1)))

        plt.xlabel('Thresholds')
        plt.ylabel('Error Rates (%)')
        plt.title('Equal Error Rate')
        plt.legend()
        plt.show()
        
        return(eer)

class LSEprim(LinearModels):
    model_name = 'LSE:Prim'
    isdual = 0
    def model(self):
        alpha_lse = []
        for k in list(set(self.Y)):
            P_lse = self.X
            I_lse = np.eye(P_lse.shape[1])
            b_lse = 10**(-4)
            y_lse = (self.Y==k).astype('int')

            ak_lse = np.dot(np.dot(np.linalg.pinv(b_lse*I_lse + P_lse.T.dot(P_lse)),P_lse.T),y_lse)
            alpha_lse.append(ak_lse) 
        
        return(np.array(alpha_lse).T)

class LSEdual(LinearModels):
    model_name = 'LSE:Dual'
    isdual = 1
    def model(self):
        beta_lsed = []
        for k in list(set(self.Y)):
            P_lsed = self.X
            I_lsed = np.eye(P_lsed.shape[0])
            b_lsed = 10**(-4)
            y_lsed = (self.Y==k).astype('int')

            bk_lsed = np.dot(np.linalg.pinv(np.dot(P_lsed,P_lsed.T) + b_lsed*I_lsed),y_lsed)
            beta_lsed.append(bk_lsed)

        return(np.array(beta_lsed).T)

class TERprim(LinearModels):
    model_name = 'TER:Prim'
    isdual = 0
        
    M_dict = {}
    M_dict['0'] = (0,1),(0,1)
    M_dict['1'] = (1,0),(1,0)
    M_dict['2'] = (1,0),(0,1)
    M_dict['3'] = (0,1),(1,0)
    M_dict['4'] = (1,0),(0.5,0)
    M_dict['5'] = (0.5,0),(1,0)
    M_dict['6'] = (1,0),(0.7,0)
    M_dict['7'] = (0.7,0),(1,0)
    M_dict['8'] = (1,0),(0.3,0)
    M_dict['9'] = (0.3,0),(1,0)
    
    def __init__(self,data,r,n,m_pos):
        super(TERprim, self).__init__(data)
        self.r = r
        self.n = n
        #self.mod_w = TERprim.M_dict[str(m_pos)]
        self.mod_w = TERprim.M_dict['1']
    def model(self):
        #simplified version of TER
        alpha = []
        for k in list(set(self.Y)):
            P = self.X
            b_ter = 10**(-4)
            I_ter = np.eye(P.shape[1])

            mk_n = self.X[self.Y!=k].shape[0]
            mk_p = self.X[self.Y==k].shape[0]

            w_n = self.mod_w[0][0] * 1/mk_n + self.mod_w[0][1]
            w_p = self.mod_w[1][0] * 1/mk_p + self.mod_w[1][1]

            ones_mkn = 1*(self.Y!=k) *w_n
            ones_mkp = 1*(self.Y==k) *w_p

            W = np.zeros((len(self.Y), len(self.Y)), float)
            np.fill_diagonal(W,ones_mkn+ones_mkp)
            yk = ((self.r-self.n)*ones_mkn+(self.r+self.n)*ones_mkp).T
            ak = np.linalg.pinv(b_ter*I_ter + (P.T).dot(W).dot(P)).dot(P.T).dot(W).dot(yk)

            alpha.append(ak)
        return(np.array(alpha).T)

class TERdual(LinearModels):
    model_name = 'TER:Dual'
    isdual = 1
    
    M_dict = {}
    M_dict['0'] = (0,1),(0,1)
    M_dict['1'] = (1,0),(1,0)
    M_dict['2'] = (1,0),(0,1)
    M_dict['3'] = (0,1),(1,0)
    M_dict['4'] = (1,0),(0.5,0)
    M_dict['5'] = (0.5,0),(1,0)
    M_dict['6'] = (1,0),(0.7,0)
    M_dict['7'] = (0.7,0),(1,0)
    M_dict['8'] = (1,0),(0.3,0)
    M_dict['9'] = (0.3,0),(1,0)
    
    def __init__(self,data,r,n,m_pos):
        super(TERdual, self).__init__(data)
        self.r = r
        self.n = n
        #self.mod_w = TERdual.M_dict[str(m_pos)]
        self.mod_w = TERdual.M_dict['1']
    def model(self):
        #TER
        beta = []
        for k in list(set(self.Y)):
            P = self.X
            b_ter = 10**(-4)
            I_ter = np.eye(P.shape[0])

            mk_n = self.X[self.Y!=k].shape[0]
            mk_p = self.X[self.Y==k].shape[0]

            w_n = self.mod_w[0][0] * 1/mk_n + self.mod_w[0][1]
            w_p = self.mod_w[1][0] * 1/mk_p + self.mod_w[1][1]

            ones_mkn = 1*(self.Y!=k) *w_n
            ones_mkp = 1*(self.Y==k) *w_p

            W = np.zeros((len(self.Y), len(self.Y)), float)
            np.fill_diagonal(W,ones_mkn+ones_mkp)
            yk = ((self.r-self.n)*ones_mkn+(self.r+self.n)*ones_mkp).T
            
            bk = np.linalg.pinv(P.dot(P.T).dot(W)+b_ter*I_ter).dot(yk)
            
            beta.append(bk)
        return(np.array(beta).T)

class SVMlinear(LinearModels):
    model_name = 'SVM:Linear'
    isdual = 0 

    def train(self,c):
        t1 = time.time()
        clf = SVC(kernel='linear',C=c,probability=True)
        clf.fit(self.X,self.Y)
        prob_mat = clf.predict_proba(self.Xval)
        
        LinearModels.train_time = time.time()-t1
        print("Training Time: " + str(LinearModels.train_time))
        return(prob_mat)
    
    def val_dist(self,val_mat):
        res_list = []
        for i in range(val_mat.shape[0]):
            for ti in range(val_mat.shape[0]):
                val_dist = val_mat[i,:] - val_mat[ti,:]
                val_score = np.sqrt(np.sum(np.multiply(val_dist,val_dist)))
                grd_truth =  int(self.Yval[i] == self.Yval[ti])
                score_list = [i,ti,grd_truth,val_score]
                res_list.append(score_list)
        
        res_df = pd.DataFrame(res_list,columns=['i','ti','truth','score'])
        #return(res_df[res_df.i != res_df.ti])
        return(val_mat)

class SVMrbf(LinearModels):
    model_name = 'SVM:RBF'
    isdual = 0 
            
    def train(self,c,gamma):
        t1 = time.time()
        clf = SVC(kernel='rbf',C=c,gamma=gamma,probability=True)
        clf.fit(self.X,self.Y)
        prob_mat = clf.predict_proba(self.Xval)
        
        LinearModels.train_time = time.time()-t1
        print("Training Time: " + str(LinearModels.train_time))
        return(prob_mat)
    
    def val_dist(self,val_mat):
        res_list = []
        for i in range(val_mat.shape[0]):
            for ti in range(val_mat.shape[0]):
                val_dist = val_mat[i,:] - val_mat[ti,:]
                val_score = np.sqrt(np.sum(np.multiply(val_dist,val_dist)))
                grd_truth =  int(self.Yval[i] == self.Yval[ti])
                score_list = [i,ti,grd_truth,val_score]
                res_list.append(score_list)
        
        res_df = pd.DataFrame(res_list,columns=['i','ti','truth','score'])
        #return(res_df[res_df.i != res_df.ti])
        return(val_mat)

class SVMpoly(LinearModels):
    model_name = 'SVM:Poly'
    isdual = 0 
            
    def train(self,c,degree):
        t1 = time.time()
        clf = SVC(kernel='poly',C=c,degree=degree,probability=True)
        clf.fit(self.X,self.Y)
        prob_mat = clf.predict_proba(self.Xval)
        
        LinearModels.train_time = time.time()-t1
        print("Training Time: " + str(LinearModels.train_time))
        return(prob_mat)
    
    def val_dist(self,val_mat):
        res_list = []
        for i in range(val_mat.shape[0]):
            for ti in range(val_mat.shape[0]):
                val_dist = val_mat[i,:] - val_mat[ti,:]
                val_score = np.sqrt(np.sum(np.multiply(val_dist,val_dist)))
                grd_truth =  int(self.Yval[i] == self.Yval[ti])
                score_list = [i,ti,grd_truth,val_score]
                res_list.append(score_list)
        
        res_df = pd.DataFrame(res_list,columns=['i','ti','truth','score'])
        #return(res_df[res_df.i != res_df.ti])
        return(val_mat)

In [4]:
## Main

# Prep Method
def PCA_HC(prep_xs,c,s,num_eig):
    M = c*s
    X = prep_xs.T
    ones_vec = np.ones([M,1])
    u = (1/M * X).dot(ones_vec)
    A = X - u.dot(ones_vec.T)
    Z = A.dot(A.T)
    W,V= np.linalg.eigh(Z)
    return(V[:,-num_eig:].T,u)

def RMmodel(X,order):
    m,l = X.shape
    M1 = []
    M2 = []
    M3 = []
    MM1 = []
    MM3 = []
    Msum = np.sum(X,axis=1)
    for i in range(order):
        for k in range(l):
            M1.append(X[:,k]**(i+1))
            if (i>0):
                M3.append(X[:,k]*Msum**(i)) 
        M2.append(Msum**(i+1))
        MM1.append(M1)
        if (i>0):
            MM3.append(M3)
    MM1 = np.array(MM1).T
    MM1 = MM1.reshape((m,-1,1)).squeeze(axis=2)
    M2 = np.array(M2).T
    if (len(MM3)):
        MM3 = np.array(MM3).T
        MM3 = MM3.reshape((m,-1,1)).squeeze(axis=2)
        P = np.concatenate((np.ones((m,1)),MM1,M2,MM3),axis=1)
    else : P = np.concatenate((np.ones((m,1)),MM1,M2),axis=1)
    return(P)


In [5]:

PATH = "D:\\Data\\WIFI\\180_100\\"
# save path
res_path = 'D:\\Git\\MILab\\190522_Wisig_KAR\\res\\'


col_df = (['timenow','host','model_type','rseed',
'splits','n_val','fc_size','batch_size','n_iter','alpha','lr','eer','trn_time','params','desc'])


res_df = pd.DataFrame()
roc_df = pd.DataFrame()

desc = 'mining_linear'

types = [0]
seeds = [10,20,30,40,50]

In [6]:
rseed=seeds[0]
splits = 0
np.random.seed(rseed)        
n_splits = 2
data_skf = ReadData(PATH,n_splits,rseed=rseed)

loading data from D:\Data\WIFI\180_100\


In [7]:
data_list = data_skf[splits]

In [8]:
                kar_x = np.squeeze(data_list[0]).reshape([-1,500*30*6])
                kar_labels = [get_key(i,data_list[1]) for i in range(len(kar_x))]
                kar_y = to_categorical(kar_labels)

                kar_xval = np.squeeze(data_list[2]).reshape([-1,500*30*6])
                kar_labels_val = [get_key(i,data_list[3]) for i in range(len(kar_xval))]
                kar_yval = to_categorical(kar_labels_val)

In [9]:
kar = KARnet([kar_x,kar_y,kar_xval,kar_yval],h=[1024,16],rseed=rseed)

In [10]:
kar.train()

In [11]:
        X_d,c_d,Xval_d,cval_d,Y_d,Yval_d = data_list

In [12]:
    def dist_kar():
        kar_x = np.squeeze(X_d).reshape([-1,500*30*6])
        bias = np.ones([kar_x.shape[0],1])
        x_calc = kar_x
        #for i in range(len(self.kar.W)-1):
        for i in range(len(kar.W)):
            x_calc = 1*kar.f(np.hstack([bias,x_calc]).dot(kar.W[i]))
        kar_fc = x_calc
        return(kar_fc)

In [13]:
kar_fc = dist_kar()

In [14]:
kar_fc.shape

(2000, 100)

In [16]:
X_d.shape

(2000, 1, 500, 30, 6)

In [19]:
X_dm = np.mean(X_d,axis=3).reshape([-1,500*6])
Xval_dm = np.mean(Xval_d,axis=3).reshape([-1,500*6])

In [34]:
            #SVM Linear
            m_svml = SVMlinear([X_dm,Y_d,Xval_dm,Yval_d])
            dist_svml = m_svml.val_dist(m_svml.train()) 

TypeError: train() missing 1 required positional argument: 'c'

In [32]:
dist_svml.shape

(2000, 100)

In [33]:
kar_fc.shape

(2000, 100)